<a href="https://colab.research.google.com/github/supportchelsea/Weather-Competition/blob/master/cf_knn_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install surprise

     |████████████████████████████████| 3.3MB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
import surprise
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from tqdm import tqdm_notebook
import sys
import os
from matplotlib import pyplot as plt

%matplotlib inline

In [0]:
path = './gdrive/My Drive/GitHub/Weather Competition/'

In [0]:
hnb = pd.read_csv(path+'data/korea_hnb.csv').iloc[:,1:]

In [0]:
hnb.columns = list(i.split('.')[1] for i in hnb.columns)

In [0]:
# hnb.head()

,bor_nm,sale_dt,gen_cd,age_cd,category,qty
0,종로구,20160101,F,20~39,립컬러,73
1,종로구,20160101,F,20~39,립케어,219
2,종로구,20160101,F,20~39,마스크팩,657
3,종로구,20160101,F,20~39,바디로션,328
4,종로구,20160101,F,20~39,체중조절,109


In [0]:
segmentation = pd.DataFrame({'segmentation' : [hnb['bor_nm'][i]+' '+hnb['gen_cd'][i]+' '+hnb['age_cd'][i] for i in range(len(hnb))]})

In [0]:
hnb = pd.concat([hnb['sale_dt'],segmentation,hnb['category'],hnb['qty']], axis=1) 

In [0]:
# df_hnb.head()

In [0]:
rainy = [20180122.0,
 20180223.0,
 20180228.0,
 20180301.0,
 20180304.0,
 20180305.0,
 20180308.0,
 20180315.0,
 20180318.0,
 20180319.0,
 20180321.0,
 20180404.0,
 20180405.0,
 20180406.0,
 20180408.0,
 20180410.0,
 20180414.0,
 20180422.0,
 20180423.0,
 20180424.0,
 20180502.0,
 20180503.0,
 20180506.0,
 20180512.0,
 20180516.0,
 20180517.0,
 20180518.0,
 20180522.0,
 20180523.0,
 20180529.0,
 20180530.0,
 20180609.0,
 20180610.0,
 20180611.0,
 20180614.0,
 20180615.0,
 20180626.0,
 20180628.0,
 20180629.0,
 20180630.0,
 20180701.0,
 20180702.0,
 20180703.0,
 20180705.0,
 20180709.0,
 20180710.0,
 20180711.0,
 20180728.0,
 20180805.0,
 20180806.0,
 20180809.0,
 20180812.0,
 20180813.0,
 20180815.0,
 20180821.0,
 20180823.0,
 20180824.0,
 20180826.0,
 20180827.0,
 20180828.0,
 20180829.0,
 20180830.0,
 20180903.0,
 20180906.0,
 20180907.0,
 20180915.0,
 20180916.0,
 20180920.0,
 20180921.0,
 20180922.0,
 20181005.0,
 20181006.0,
 20181010.0,
 20181023.0,
 20181026.0,
 20181028.0,
 20181107.0,
 20181108.0,
 20181121.0,
 20181124.0,
 20181203.0,
 20181204.0]

In [0]:
rainy = [str(i).split('.')[0] for i in rainy]

In [0]:
rainy_date = []

for i in rainy:
  for j in hnb['sale_dt'] :
    if i == str(j):
      rainy_date.append(True)
    else:
      rainy_date.append(False)

In [0]:
# len(rainy_date)

In [0]:
df_rainy = pd.DataFrame(rainy_date)

df_rainy.columns = ['rainy_day']

In [0]:
df_hnb = pd.concat([hnb,df_rainy],axis=1)

In [0]:
df_hnb = df_hnb[df_hnb['sale_dt']>20171231]

In [0]:
rainy = df_hnb[df_hnb['rainy_day']==True].iloc[:,1:4].dropna() #비오는 날만

not_rainy = df_hnb[df_hnb['rainy_day']==False].iloc[:,1:4]

In [0]:
not_rainy.head(3)

,segmentation,category,qty
547246,종로구 F 20~39,네일,36.0
547247,종로구 F 20~39,립컬러,182.0
547248,종로구 F 20~39,립케어,146.0


In [0]:
len(rainy)

592

In [0]:
not_rainy.head(3)
len(not_rainy)

272436

In [0]:
rainy.head(3)

,segmentation,category,qty
547479,종로구 F 00~19,립케어,109.0
547480,종로구 F 20~39,네일,36.0
547481,종로구 F 20~39,립컬러,36.0


In [0]:
# rainy = [str(i).split('.')[0] for i in rainy]

## Rainy

In [0]:
l_qty = rainy['qty'].min()
u_qty = rainy['qty'].max()

print('Quantity range : {0} to {1}'.format(l_qty,u_qty))

Quantity range : 36.0 to 6567.0


In [0]:
reader = surprise.Reader()

In [0]:
surprise_rainy = surprise.Dataset.load_from_df(rainy,reader)
# df_tmp2 = surprise.Dataset.load_from_df(tmp2,reader)

In [0]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(surprise_rainy, test_size=0.25,random_state=42)

In [0]:
sim_options = {'name': 'cosine',
               'user_based': False , # compute  similarities between items
               'min_support':2,
               'random_state':42
               }

algo_knn = surprise.KNNBasic(sim_options=sim_options)

# algo_SVDpp = surprise.SVDpp(user_based= False,random_state=42)

In [0]:
algo_knn.fit(trainset)
# algo_SVDpp.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
sim_knn = algo_knn.compute_similarities()
# sim_SVDpp = algo_SVDpp.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
# algo_SVDpp.sim_options

{'user_based': True}

In [0]:
knn_rainy = pd.DataFrame(sim_knn)
# SVDpp_rainy = pd.DataFrame(sim_SVDpp)

In [0]:
label = hnb['category'].unique()

In [0]:
knn_rainy.columns = label
knn_rainy.index  = label
knn_rainy

,립컬러,립케어,마스크팩,바디로션,체중조절,크림로션,훼이셜클렌저,선케어,네일,제모제
립컬러,1.000000,0.859883,0.912462,0.832488,0.903779,0.852938,0.688625,0.767230,0.781112,0.807020
립케어,0.859883,1.000000,0.888123,0.906501,0.879121,0.805071,0.783195,0.874651,0.896595,0.608505
마스크팩,0.912462,0.888123,1.000000,0.904986,0.895806,0.852721,0.704597,0.552387,0.829305,0.656714
바디로션,0.832488,0.906501,0.904986,1.000000,0.924282,0.860022,0.487415,0.853207,0.633229,0.546374
체중조절,0.903779,0.879121,0.895806,0.924282,1.000000,0.897915,0.782247,0.868015,0.773114,0.608161
크림로션,0.852938,0.805071,0.852721,0.860022,0.897915,1.000000,0.698607,0.956617,0.833076,0.616689
훼이셜클렌저,0.688625,0.783195,0.704597,0.487415,0.782247,0.698607,1.000000,0.980993,0.884968,0.865503
선케어,0.767230,0.874651,0.552387,0.853207,0.868015,0.956617,0.980993,1.000000,0.961068,0.982406
네일,0.781112,0.896595,0.829305,0.633229,0.773114,0.833076,0.884968,0.961068,1.000000,0.877449
제모제,0.807020,0.608505,0.656714,0.546374,0.608161,0.616689,0.865503,0.982406,0.877449,1.000000


In [0]:
# SVDpp.columns = label
# SVDpp.index = label
# SVDpp.

---
# Not Rainy

In [0]:
surprise_not_rainy = surprise.Dataset.load_from_df(not_rainy,reader)

trainset, testset = train_test_split(surprise_not_rainy, test_size=0.25,shuffle=True,random_state=42)

algo_knn.fit(trainset)

sim_knn = algo_knn.compute_similarities()

knn_not_rainy = pd.DataFrame(sim_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
knn_not_rainy.columns = label
knn_not_rainy.index  = label
knn_not_rainy

,네일,립컬러,립케어,마스크팩,바디로션,크림로션,훼이셜클렌저,선케어,체중조절,제모제
네일,1.000000,0.665876,0.641221,0.633496,0.268096,0.568624,0.361723,0.602148,0.619832,0.625034
립컬러,0.665876,1.000000,0.702688,0.696861,0.272874,0.636336,0.376789,0.654483,0.679829,0.691434
립케어,0.641221,0.702688,1.000000,0.668609,0.263598,0.620537,0.353243,0.624909,0.660051,0.667010
마스크팩,0.633496,0.696861,0.668609,1.000000,0.324052,0.594840,0.433504,0.648138,0.656914,0.664077
바디로션,0.268096,0.272874,0.263598,0.324052,1.000000,0.251350,0.150086,0.240647,0.266883,0.282727
크림로션,0.568624,0.636336,0.620537,0.594840,0.251350,1.000000,0.336903,0.577517,0.592199,0.587775
훼이셜클렌저,0.361723,0.376789,0.353243,0.433504,0.150086,0.336903,1.000000,0.347526,0.365784,0.427517
선케어,0.602148,0.654483,0.624909,0.648138,0.240647,0.577517,0.347526,1.000000,0.620760,0.645575
체중조절,0.619832,0.679829,0.660051,0.656914,0.266883,0.592199,0.365784,0.620760,1.000000,0.652728
제모제,0.625034,0.691434,0.667010,0.664077,0.282727,0.587775,0.427517,0.645575,0.652728,1.000000


---
# Temperature

In [0]:
high_temp = [20180802.0, 20180801.0, 20180803.0, 20180731.0, 20180730.0, 20180815.0, 20180814.0, 20180729.0, 20180723.0, 20180724.0, 20180722.0, 20180804.0, 20180808.0, 20180727.0, 20180810.0, 20180807.0, 20180813.0, 20180811.0, 20180812.0, 20180806.0] 
low_temp = [20180203.0, 20181209.0, 20181229.0, 20180110.0, 20181230.0, 20181208.0, 20180129.0, 20180207.0, 20180205.0, 20180204.0, 20181227.0, 20180206.0, 20180112.0, 20181228.0, 20180127.0, 20180111.0, 20180123.0, 20180125.0, 20180124.0, 20180126.0]

In [0]:
high_temp = [str(i).split('.')[0] for i in high_temp]
low_temp = [str(i).split('.')[0] for i in low_temp]

In [0]:
hnb.head(3)

,sale_dt,segmentation,category,qty
0,20160101,종로구 F 20~39,립컬러,73
1,20160101,종로구 F 20~39,립케어,219
2,20160101,종로구 F 20~39,마스크팩,657


In [0]:
high_temp_date = []

for i in high_temp:
  for j in hnb['sale_dt'] :
    if i == str(j):
      high_temp_date.append(True)
    else:
      high_temp_date.append(False)

In [0]:
low_temp_date = []

for i in low_temp:
  for j in hnb['sale_dt'] :
    if i == str(j):
      low_temp_date.append(True)
    else:
      low_temp_date.append(False)

In [0]:
high_temp = pd.DataFrame(high_temp_date,columns=['high_temp'])
low_temp = pd.DataFrame(low_temp_date,columns=['low_temp'])

In [0]:
df_high_temp = pd.concat([hnb,high_temp],axis=1)
df_low_temp = pd.concat([hnb,low_temp],axis=1)


In [0]:
df_high_temp.head(3)

,sale_dt,segmentation,category,qty,high_temp
0,20160101.0,종로구 F 20~39,립컬러,73.0,False
1,20160101.0,종로구 F 20~39,립케어,219.0,False
2,20160101.0,종로구 F 20~39,마스크팩,657.0,False


In [0]:
df_high_temp = df_high_temp[df_high_temp['high_temp']==True]
df_low_temp = df_low_temp[df_low_temp['low_temp']==True]

## High Temperature

In [0]:
df_high_temp.head(3)

,sale_dt,segmentation,category,qty,high_temp
549164,20180802.0,종로구 F 20~39,네일,73.0,True
549165,20180802.0,종로구 F 20~39,립컬러,182.0,True
549166,20180802.0,종로구 F 20~39,립케어,36.0,True


In [0]:
df_high_temp = df_high_temp[df_high_temp['sale_dt'] > 20171231].iloc[:,[1,2,3]]

df_high_temp.head(3)

,segmentation,category,qty
549164,종로구 F 20~39,네일,73.0
549165,종로구 F 20~39,립컬러,182.0
549166,종로구 F 20~39,립케어,36.0


In [0]:
from surprise.model_selection import train_test_split


In [0]:
surprise_df_high_temp = surprise.Dataset.load_from_df(df_high_temp,reader)

trainset, testset = train_test_split(surprise_df_high_temp, test_size=0.25,shuffle=True,random_state=42)

algo_knn.fit(trainset)

sim_knn = algo_knn.compute_similarities()

knn_high_temp = pd.DataFrame(sim_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
knn_high_temp.columns = label
knn_high_temp.index  = label
knn_high_temp

,립컬러,립케어,마스크팩,바디로션,체중조절,크림로션,훼이셜클렌저,선케어,네일,제모제
립컬러,1.000000,0.697875,0.919853,0.896806,0.866130,0.868510,0.703496,0.860351,0.840062,0.878544
립케어,0.697875,1.000000,0.871898,0.893655,0.838694,0.825543,0.864314,0.752856,0.831684,0.903387
마스크팩,0.919853,0.871898,1.000000,0.938108,0.883366,0.846419,0.898995,0.797587,0.832069,0.916970
바디로션,0.896806,0.893655,0.938108,1.000000,0.885084,0.876047,0.811365,0.911900,0.849388,0.922658
체중조절,0.866130,0.838694,0.883366,0.885084,1.000000,0.883846,0.887205,0.915350,0.890233,0.977132
크림로션,0.868510,0.825543,0.846419,0.876047,0.883846,1.000000,0.870693,0.909095,0.929383,0.863849
훼이셜클렌저,0.703496,0.864314,0.898995,0.811365,0.887205,0.870693,1.000000,0.852422,0.762444,0.957855
선케어,0.860351,0.752856,0.797587,0.911900,0.915350,0.909095,0.852422,1.000000,0.856229,0.955706
네일,0.840062,0.831684,0.832069,0.849388,0.890233,0.929383,0.762444,0.856229,1.000000,0.932196
제모제,0.878544,0.903387,0.916970,0.922658,0.977132,0.863849,0.957855,0.955706,0.932196,1.000000


## Lower Temperature

In [0]:
df_low_temp = df_low_temp[df_low_temp['sale_dt'] > 20171231].iloc[:,[1,2,3]]

df_low_temp.head(3)

,segmentation,category,qty
547614,종로구 F 20~39,립컬러,73.0
547615,종로구 F 20~39,립케어,146.0
547616,종로구 F 20~39,마스크팩,182.0


In [0]:
surprise_df_low_temp = surprise.Dataset.load_from_df(df_low_temp,reader)

trainset, testset = train_test_split(surprise_df_low_temp, test_size=0.25,shuffle=True,random_state=42)

algo_knn.fit(trainset)

sim_knn = algo_knn.compute_similarities()

knn_low_temp = pd.DataFrame(sim_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
knn_low_temp.columns = label
knn_low_temp.index  = label
knn_low_temp

,립컬러,립케어,마스크팩,바디로션,체중조절,크림로션,훼이셜클렌저,선케어,네일,제모제
립컬러,1.000000,0.782011,0.903615,0.893575,0.746064,0.927045,0.828198,0.840064,0.947496,0.756092
립케어,0.782011,1.000000,0.825328,0.806023,0.763134,0.864666,0.750501,0.893724,0.815335,0.773718
마스크팩,0.903615,0.825328,1.000000,0.907933,0.884241,0.918157,0.827688,0.815211,0.943601,0.828459
바디로션,0.893575,0.806023,0.907933,1.000000,0.861255,0.936566,0.816072,0.795223,0.937307,0.771183
체중조절,0.746064,0.763134,0.884241,0.861255,1.000000,0.797446,0.866269,0.758342,0.811439,0.781990
크림로션,0.927045,0.864666,0.918157,0.936566,0.797446,1.000000,0.745525,0.878115,0.914096,0.825610
훼이셜클렌저,0.828198,0.750501,0.827688,0.816072,0.866269,0.745525,1.000000,0.686080,0.901587,0.736339
선케어,0.840064,0.893724,0.815211,0.795223,0.758342,0.878115,0.686080,1.000000,0.822323,0.942082
네일,0.947496,0.815335,0.943601,0.937307,0.811439,0.914096,0.901587,0.822323,1.000000,0.748824
제모제,0.756092,0.773718,0.828459,0.771183,0.781990,0.825610,0.736339,0.942082,0.748824,1.000000


---
# Humid

In [0]:
upper_humid = [20180517.0, 20180701.0, 20180828.0, 20180702.0, 20181108.0, 20180516.0, 20180827.0, 20180315.0, 20180423.0, 20180512.0, 20181203.0, 20180921.0, 20180626.0, 20180824.0, 20180502.0, 20180710.0, 20180405.0, 20180829.0, 20180304.0, 20180506.0] 
lower_humid = [20180128.0, 20181128.0, 20181226.0, 20180205.0, 20180206.0, 20180412.0, 20180126.0, 20181228.0, 20181208.0, 20181207.0, 20181209.0, 20180125.0, 20180221.0, 20180218.0, 20180124.0, 20181229.0, 20180215.0, 20180413.0, 20180217.0, 20181227.0]

In [0]:
upper_humid = [str(i).split('.')[0] for i in upper_humid]
lower_humid = [str(i).split('.')[0] for i in lower_humid]

In [0]:
upper_humid_date = []

for i in upper_humid:
  for j in hnb['sale_dt'] :
    if i == str(j):
      upper_humid_date.append(True)
    else:
      upper_humid_date.append(False)

In [0]:
lower_humid_date = []

for i in lower_humid:
  for j in hnb['sale_dt'] :
    if i == str(j):
      lower_humid_date.append(True)
    else:
      lower_humid_date.append(False)

In [0]:
upper_humid = pd.DataFrame(upper_humid_date,columns=['upper_humid'])
lower_humid = pd.DataFrame(lower_humid_date,columns=['lower_humid'])

In [0]:
df_upper_humid= pd.concat([hnb,upper_humid],axis=1)

df_lower_humid = pd.concat([hnb,lower_humid],axis=1)


In [0]:
df_upper_humid = df_upper_humid[df_upper_humid['upper_humid']==True]
df_lower_humid = df_lower_humid[df_lower_humid['lower_humid']==True]

## Upper Humid

In [0]:
df_upper_humid = df_upper_humid[df_upper_humid['sale_dt'] > 20171231].iloc[:,[1,2,3]]

df_upper_humid.head(3)

,segmentation,category,qty
548507,종로구 F 20~39,네일,36.0
548508,종로구 F 20~39,립컬러,182.0
548509,종로구 F 20~39,마스크팩,73.0


In [0]:
surprise_df_upper_humid = surprise.Dataset.load_from_df(df_upper_humid,reader)

trainset, testset = train_test_split(surprise_df_upper_humid, test_size=0.25,shuffle=True,random_state=42)

algo_knn.fit(trainset)

sim_knn = algo_knn.compute_similarities()

knn_upper_humid = pd.DataFrame(sim_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
knn_upper_humid.columns = label
knn_upper_humid.index  = label
knn_upper_humid

,립컬러,립케어,마스크팩,바디로션,체중조절,크림로션,훼이셜클렌저,선케어,네일,제모제
립컬러,1.000000,0.899465,0.707453,0.883083,0.826419,0.847264,0.696581,0.587703,0.720310,0.790291
립케어,0.899465,1.000000,0.775756,0.928536,0.878105,0.876171,0.842711,0.736496,0.662591,0.869485
마스크팩,0.707453,0.775756,1.000000,0.818910,0.775335,0.931264,0.851393,0.972451,0.818420,0.894680
바디로션,0.883083,0.928536,0.818910,1.000000,0.905326,0.921371,0.862606,0.716728,0.777649,0.802208
체중조절,0.826419,0.878105,0.775335,0.905326,1.000000,0.839258,0.813329,0.791758,0.611217,0.899376
크림로션,0.847264,0.876171,0.931264,0.921371,0.839258,1.000000,0.827549,0.764403,0.937877,0.882934
훼이셜클렌저,0.696581,0.842711,0.851393,0.862606,0.813329,0.827549,1.000000,0.761309,0.761719,0.905216
선케어,0.587703,0.736496,0.972451,0.716728,0.791758,0.764403,0.761309,1.000000,0.897675,0.801933
네일,0.720310,0.662591,0.818420,0.777649,0.611217,0.937877,0.761719,0.897675,1.000000,0.657140
제모제,0.790291,0.869485,0.894680,0.802208,0.899376,0.882934,0.905216,0.801933,0.657140,1.000000


## Lower Humid

In [0]:
df_lower_humid = df_lower_humid[df_lower_humid['sale_dt'] > 20171231].iloc[:,[1,2,3]]

df_lower_humid.head(3)

,segmentation,category,qty
547542,종로구 F 20~39,네일,109.0
547543,종로구 F 20~39,립컬러,511.0
547544,종로구 F 20~39,립케어,219.0


In [0]:
surprise_df_lower_humid = surprise.Dataset.load_from_df(df_lower_humid,reader)

trainset, testset = train_test_split(surprise_df_lower_humid, test_size=0.25,shuffle=True,random_state=42)

algo_knn.fit(trainset)

sim_knn = algo_knn.compute_similarities()

knn_lower_humid = pd.DataFrame(sim_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
knn_lower_humid.columns = label
knn_lower_humid.index  = label
knn_lower_humid

,립컬러,립케어,마스크팩,바디로션,체중조절,크림로션,훼이셜클렌저,선케어,네일,제모제
립컬러,1.000000,0.885182,0.751261,0.937856,0.915215,0.877002,0.880567,0.803869,0.853312,0.857150
립케어,0.885182,1.000000,0.815737,0.928136,0.913567,0.888369,0.804282,0.741175,0.844933,0.739243
마스크팩,0.751261,0.815737,1.000000,0.842463,0.879810,0.738091,0.721290,0.731039,0.889134,0.861790
바디로션,0.937856,0.928136,0.842463,1.000000,0.872303,0.901282,0.850125,0.838351,0.801571,0.989092
체중조절,0.915215,0.913567,0.879810,0.872303,1.000000,0.894260,0.869619,0.801909,0.906347,0.909751
크림로션,0.877002,0.888369,0.738091,0.901282,0.894260,1.000000,0.926960,0.773796,0.860852,0.836339
훼이셜클렌저,0.880567,0.804282,0.721290,0.850125,0.869619,0.926960,1.000000,0.774550,0.867930,0.842190
선케어,0.803869,0.741175,0.731039,0.838351,0.801909,0.773796,0.774550,1.000000,0.853642,0.826866
네일,0.853312,0.844933,0.889134,0.801571,0.906347,0.860852,0.867930,0.853642,1.000000,0.829285
제모제,0.857150,0.739243,0.861790,0.989092,0.909751,0.836339,0.842190,0.826866,0.829285,1.000000


In [0]:
(knn_high_temp)

In [0]:
knn_high_temp.knn_high_temp.as_matrix
knn_low_temp
knn_lower_humid
knn_upper_humid
knn_rainy

In [0]:
#! /usr/bin/env python
"""
==============
Directed Graph
==============

Draw a graph with directed edges using a colormap and different node sizes.

Edges have different colors and alphas (opacity). Drawn using matplotlib.
"""
# Author: Rodrigo Dorantes-Gilardi (rodgdor@gmail.com)

from __future__ import division
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx


In [0]:
knn_high_temp

,립컬러,립케어,마스크팩,바디로션,체중조절,크림로션,훼이셜클렌저,선케어,네일,제모제
립컬러,1.000000,0.697875,0.919853,0.896806,0.866130,0.868510,0.703496,0.860351,0.840062,0.878544
립케어,0.697875,1.000000,0.871898,0.893655,0.838694,0.825543,0.864314,0.752856,0.831684,0.903387
마스크팩,0.919853,0.871898,1.000000,0.938108,0.883366,0.846419,0.898995,0.797587,0.832069,0.916970
바디로션,0.896806,0.893655,0.938108,1.000000,0.885084,0.876047,0.811365,0.911900,0.849388,0.922658
체중조절,0.866130,0.838694,0.883366,0.885084,1.000000,0.883846,0.887205,0.915350,0.890233,0.977132
크림로션,0.868510,0.825543,0.846419,0.876047,0.883846,1.000000,0.870693,0.909095,0.929383,0.863849
훼이셜클렌저,0.703496,0.864314,0.898995,0.811365,0.887205,0.870693,1.000000,0.852422,0.762444,0.957855
선케어,0.860351,0.752856,0.797587,0.911900,0.915350,0.909095,0.852422,1.000000,0.856229,0.955706
네일,0.840062,0.831684,0.832069,0.849388,0.890233,0.929383,0.762444,0.856229,1.000000,0.932196
제모제,0.878544,0.903387,0.916970,0.922658,0.977132,0.863849,0.957855,0.955706,0.932196,1.000000


In [0]:
for i in knn_high_temp.columns():
  knn_high_temp[i]
  print(i)

립컬러
립케어
마스크팩
바디로션
체중조절
크림로션
훼이셜클렌저
선케어
네일
제모제


In [0]:
G = nx.from_scipy_sparse_matrix(a)
pos = nx.layout.spring_layout(G)

node_sizes = [3 + 10 * i for i in range(len(G))]
M = G.number_of_edges()
edge_colors = range(2, M + 2)
edge_alphas = [(5 + i) / (M + 4) for i in range(M)]

nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='blue')
edges = nx.draw_networkx_edges(G, pos, node_size=node_sizes, arrowstyle='->',
                               arrowsize=10, edge_color=edge_colors,
                               edge_cmap=plt.cm.Blues, width=2)
# set alpha value for each edge
for i in range(M):
    edges[i].set_alpha(edge_alphas[i])

pc = mpl.collections.PatchCollection(edges, cmap=plt.cm.Blues)
pc.set_array(edge_colors)
plt.colorbar(pc)

ax = plt.gca()
ax.set_axis_off()
plt.show()

AttributeError: ignored